In this jupyter-notebook we show how to implement our method to estimate two-dimensional unitary gates. The script **experiments.py** contains the functions needed to carry out this task.

In [1]:
import numpy as np
import qiskit.quantum_info as qi
import pandas as pd
import matplotlib.pyplot as plt

from qiskit.extensions import UnitaryGate
from qiskit.quantum_info.operators import Operator
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit import (
    QuantumCircuit, 
    QuantumRegister
)

from qiskit_experiments.library.tomography import ( 
    ProcessTomography, 
    MitigatedProcessTomography
)

from experiments import (
    run_experiments, 
    UnitaryEstimation,
    CustomNoiseModel
)


# from qiskit_ibm_provider import IBMProvider
# IBMProvider.save_account(token='')

service = QiskitRuntimeService()
chosen_backen = service.get_backend('ibm_nairobi')
# It is possible to chose another backend (see all available backends running service.backends()).

noise_model = NoiseModel.from_backend(chosen_backen)

%load_ext autoreload
%autoreload 2

### Define the model

In the following lines one can chose wether *noise*, from the `noise_model` variable, is included in the model or not by setting `noisy` to `True`. Also, one can choose to apply *readout mitigation* by setting `rout_mitigation` to `True`.

In [2]:
noisy = True # With True, noise_model is applied in the experiment 
rout_mitigation = True  #Apply readout mitigation

if noisy:
    backend = AerSimulator(noise_model=noise_model)
else:
    backend = AerSimulator()

#Chose a unitary gate at random. This is the one to be estimated
target_unitary = qi.random_unitary(2).data 

#Create a qiskit.extensions.unitary.UnitaryGate instance from target_unitary
target_gate = UnitaryGate(target_unitary) 


### Perform measurements and estimate the unknown `target_gate`

In [3]:
# Select the size of the ensemble 
num_of_shots = 1000

# Perform measurements
pd1,pd2,pd3 = run_experiments(target_gate, 
                            backend, 
                            num_of_shots = num_of_shots, 
                            rout_mitigation = rout_mitigation)

# Estimate the unitary gate
estimated_unitary = UnitaryEstimation(pd1,pd2,pd3)

# Compute the choi representation of "estimated_unitary"
choi_miro = qi.Choi(Operator(estimated_unitary))

### Compare the estimated gate with the target gate using the *averagre gate fidelity*

https://qiskit.org/documentation/stable/0.19/stubs/qiskit.quantum_info.average_gate_fidelity.html

In [4]:
# Compute the fidelity 
fmiro = qi.average_gate_fidelity(choi_miro, target=Operator(target_unitary))

print(f"target_unitary was estimated with fidelity {fmiro}")

target_unitary was estimated with fidelity 0.9942080226519806


### Customize noise model
In the next cell the noise model is customized. For example, let us estimate `target_unitary` considering an ideal **cnot** gate (`cx = 0`). Another values for errors affecting single qubit gates, conditional gates, thermal relaxation, and measurements are set.

In [12]:
t1 = 110e3    #110e3 nanoseconds
t2 = 147e3   #147e3 nanoseconds
sx_e = 0.00045
rdt = 0.0841   #0.0841
cx = 0 #0.0142

#Relaxation times(ns)
T1s = [t1 for i in range(5)]
T2s = [t2 for i in range(5)]

#Error probabilities
sx_errors = [sx_e for i in range(5)]
x_errors = [sx_e for i in range(5)]

#readout_errors = [0.0406 , 0.0444 , 0.0841 , 0.0338 , 0.0362]
readout_errors = [rdt for i in range(5)]

# Instruction times (in nanoseconds)
time_id = 0
time_sx = 50 #50  # (single X90 pulse)
time_x = 100 # (two X90 pulses)
time_cx = 300
time_reset = 1000  # 1 microsecond
time_measure = 1000 # 1 microsecond

#arguments for function
cxs = [cx for i in range(4)]

Rel_Ts = [T1s,T2s]
Errores = [sx_errors,x_errors,cxs,readout_errors]
Times = [time_id,time_sx,time_x,time_cx,time_reset,time_measure]

customized_noise = CustomNoiseModel(Rel_Ts,Errores,Times)

In [13]:
num_of_shots = 1000

rout_mitigation = True  

cpd1,cpd2,cpd3 = run_experiments(target_gate, 
                            AerSimulator(noise_model=customized_noise),  # Here the customized noise model is added
                            num_of_shots = num_of_shots, 
                            rout_mitigation = rout_mitigation)

# Estimate the unitary gate
estimated_unitary_cnoise = UnitaryEstimation(cpd1,cpd2,cpd3)

# Compute the choi representation of "estimated_unitary"
choi_miro_cnoise = qi.Choi(Operator(estimated_unitary_cnoise))

In [14]:
fmiro_cnoise = qi.average_gate_fidelity(choi_miro_cnoise, target=Operator(target_unitary))

print(f"Using the customized_noise model above, the")
print(f"target_unitary was estimated with fidelity {fmiro_cnoise}")

Using the customized_noise model above, the
target_unitary was estimated with fidelity 0.9996830728465204
